In [1]:
import pandas as pd

df = pd.read_csv("Fake_Real_Data.csv")
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [2]:
df.shape

(9900, 2)

In [3]:
df.label.value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [4]:
df["label_num"] = df["label"].map({"Fake": 0, "Real": 1})
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [5]:
import spacy 
nlp = spacy.load("en_core_web_lg")

In [6]:
df["vector"] = df["Text"].apply(lambda x: nlp(x).vector)

In [8]:
df.vector.values

array([array([-0.6759837 ,  1.4263071 , -2.318466  , -0.451093  ,  2.8199325 ,
               0.85161996,  0.01045941,  3.6876464 , -1.1069572 , -0.89137274,
               4.7383623 ,  0.93590134, -2.4224715 ,  0.1756112 ,  1.1680135 ,
               1.358259  ,  0.968191  , -1.223655  , -0.95027757, -0.3604985 ,
               0.54369414, -0.48284245, -0.42516926, -1.8079175 , -0.63552403,
              -1.7014275 , -1.5113001 , -0.5503619 , -0.78364307,  0.83594626,
               0.8964651 , -0.1389701 , -0.7893373 , -0.42614973, -2.1502306 ,
              -0.92926306, -0.8751075 ,  0.8984946 ,  1.1605583 ,  1.0838758 ,
              -0.04290289,  0.0810288 , -0.04022507,  0.16692582, -1.4921017 ,
               1.1105057 ,  0.07551349, -3.346768  , -1.3400731 ,  2.8042486 ,
              -1.1094471 ,  0.2824121 ,  0.8654462 , -3.8102863 , -0.59461933,
               0.0378601 ,  0.2941968 ,  1.0687567 ,  0.99998826,  0.66275495,
               1.6435239 , -0.40380004, -0.6515654 ,

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.vector.values, df.label_num, test_size=0.2, random_state=2022)

In [10]:
X_train.shape

(7920,)

In [11]:
X_train

array([array([-2.26311374e+00,  3.07298154e-01, -1.33053243e+00,  8.23658347e-01,
               4.81299686e+00, -6.86131325e-03,  3.35425436e-01,  3.76113081e+00,
               5.39720833e-01, -1.42519677e+00,  4.56928253e+00,  1.55305433e+00,
              -3.53332734e+00,  6.87306941e-01, -5.81286728e-01,  2.71541500e+00,
               1.39683473e+00,  9.30782378e-01, -8.92450333e-01, -1.38076222e+00,
               1.36910903e+00, -1.13511407e+00, -9.53715205e-01,  8.05722892e-01,
               6.48910582e-01, -1.65908492e+00, -2.64194083e+00, -5.77975094e-01,
              -1.42578706e-02,  2.33008236e-01,  2.95631811e-02,  2.99089730e-01,
              -4.90643919e-01, -1.33279932e+00, -2.74737763e+00, -9.77295876e-01,
              -5.31269848e-01,  1.10031247e+00,  4.24643457e-01, -3.41824055e-01,
               2.72467971e-01,  8.62076208e-02, -2.81822890e-01, -2.08477139e-01,
              -1.27764702e+00,  8.22918952e-01,  7.95027316e-01, -1.89551938e+00,
              -6

In [12]:
import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [13]:
X_train_2d

array([[-2.2631137 ,  0.30729815, -1.3305324 , ..., -1.430508  ,
        -1.3995196 ,  0.6761199 ],
       [-2.0770946 ,  0.3782537 , -2.353002  , ..., -0.513723  ,
        -2.7441313 ,  0.7244029 ],
       [-2.4900377 ,  0.48922613, -2.2653627 , ..., -1.849896  ,
        -2.1738544 ,  0.9719141 ],
       ...,
       [-2.3045952 ,  0.29060102, -1.654057  , ..., -1.8792329 ,
        -1.3021306 ,  0.48727357],
       [-2.7914732 ,  0.26514372, -1.7136743 , ..., -1.993619  ,
        -1.2461776 ,  0.9651071 ],
       [-1.061138  ,  0.81994104, -2.5229065 , ...,  0.57455444,
        -2.9443104 ,  1.168517  ]], dtype=float32)

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)

clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

MultinomialNB()

In [16]:
y_pred = clf.predict(scaled_test_embed)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1024
           1       0.94      0.95      0.94       956

    accuracy                           0.94      1980
   macro avg       0.94      0.94      0.94      1980
weighted avg       0.94      0.94      0.94      1980



In [17]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
clf.fit(X_train_2d, y_train)
y_pred = clf.predict(X_test_2d)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1024
           1       0.99      0.99      0.99       956

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

